##  Optimze and a Dense Neural Network for gap filling and feature identification

** With a few tweaks to RepRunner, an LSTM can be run instead

In [29]:
import time
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations
# from functools import partial
from multiprocessing import Pool
from functools import partial
from sklearn import metrics
# from sklearn.model_selection import train_test_split

## Personal Modules
import ReadStandardTimeFill as RSTF
import importlib
import DenseNet as Dense
import MiscFuncs as MF
importlib.reload(Dense)
importlib.reload(RSTF)
importlib.reload(MF)

# %matplotlib inline

%matplotlib notebook
%config IPCompleter.greedy=True

from scipy.optimize import minimize, curve_fit
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.externals import joblib
from matplotlib import cm

from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison

from ipywidgets import FloatProgress, HTML
from IPython.display import display, clear_output
import os  
import shutil
from keras import backend as K
try:pool.close()
except:pass


def Test(params,X,y,YScaled,XScaled,pool):
    return(np.random.rand(params['K']))


def ModSelect(Scope,Site):
    if Site == 'Illisarvik':
        if Scope == 'Full':
            Model = ['wind_speed','PPFD_Avg','AirTC_Avg','VPD',
                    'Temp','VWC','Sedge','Shrub','Grass','Upland']
        if Scope == 'Test':
            Model = ['wind_dir','wind_speed','AirTC_Avg',
                    'Temp','VWC','Sedge','Upland']
#             Model = ['Temp','VWC','Sedge','Upland','VPD','PPFD_Avg']
#         if Scope == 'Test':
#             Model = ['PPFD_Avg','Sedge','Sparse']#,'VPD','wind_speed']
    if Site == 'FishIsland':
        BaseFactors = []
        if Scope == 'Full':
            Model = ['Wind Spd','Ta','Rn','PPFD','Water Table',
            'Ts 2.5 cm','Ts 15 cm','VWC','Active Layer','24H Rain']
        if Scope == 'Test':
            Model = ['Wind Spd','Ta','Rn','PPFD','Water Table',
            'Ts 2.5 cm','Ts 15 cm','VWC','Active Layer','24H Rain']
    return(Model)

def Combos(Model,L,factor=None):
    Models=[]
    for c in combinations(Model,L):
        c = list(c)
        if factor is None:
            Models.append(c)
        else:
            for f in factor:
                f = f.split('+')
                if set(f).issubset(set(c)) and c not in Models:
                    Models.append(c)
                    
#     print('Models: ',Models)
    return(Models)

def Stats(mse,se,j,i,params):
    df = pd.DataFrame(index = [str(j)+'_'+str(i)],
                      data={'Model':[params['Model']],
                            'MSE':[mse],
                            'Size':j,
                            'Number':i,
                            'SE':[se],
                            'Performance':0})
    return(df)

def t(p,n):
    alpha = 1-p
    df = n-1
    return(stats.t.ppf(alpha,df))

FirstRun = True
# FirstRun = False

MP=True
# MP=False
Scope = 'Full'
# if Scope == Test:
#     processes=2
# else:
processes=4
    
cwd = os.getcwd()
# for Site in ['Illisarvik','FishIsland']:
Site='FishIsland'
target='fch4'
alpha = .05
print(cwd)


pd.set_option('max_colwidth',200)
def Display (tar,prog1,prog2,MdLs,MdL):
    clear_output()
    display(tar)
    display(prog1)
    display(prog2)
    display(MdLs)
    display(MdL)
    
tar = HTML(
            value=" ",
            placeholder='Target: ',
            description='Target: ',
        )
kwt = HTML(
        value=str(0),
        placeholder='Quit Score: ',
        description='Quit Score: ',
        )

C:\Users\wesle\NetworkAnalysis


# Model

In [31]:

Time = time.time()

kwit = 0
if __name__ == '__main__':
    for target in ['fco2','fch4']:
        XVarriables=ModSelect(Scope,Site)
        if FirstRun == True:
            start=1
            stop = len(XVarriables)+2
            try:shutil.rmtree(cwd+'/'+Site+'/'+target+'/')
            except:pass
            os.mkdir(cwd+'/'+Site+'/'+target+'/')
        else:
            params = Dense.Params(Scope,target,MP)
            params['Dpath'] = cwd+'/'+Site+'/'
            Records = pd.read_csv(cwd+'/'+Site+'/'+target+'_Runs.csv',delimiter = ',',header = 0,index_col=[0])
            start = 2#Records['Size'].max()+1
            Level = Records.loc[Records['Size']==start-1]
            Min = Level.loc[Level['MSE']==Level['MSE'].min()]
            Sv = (((Level['SE']**2+Min['SE'].values[0]**2)/2)**.5)
            Level['T']=(Level['MSE']-Min['MSE'].values[0])/(Sv*(2/params['K'])**.5)
            Factors = Level.loc[Level['T']<=stats.t.ppf(1-alpha,params['K']),'Model'].values
            Level.loc[Level['T']<=stats.t.ppf(1-alpha,params['K']),'Performance']=1
            
            stop = len(XVarriables)
            if start >2:
                Last = Records.loc[Records['Size']==start-2].sort_values('MSE').reset_index()
                Rec = Last.loc[Last['MSE']==Last['MSE'].min()]
                Sv = (((Level['SE']**2+Rec['SE'].values[0]**2)/2)**.5)
                Level['T2']=(Level['MSE']-Rec['MSE'].values[0])/(Sv*(2/params['K'])**.5)

                Level.loc[Level['T2']<=-stats.t.ppf(1-alpha,params['K']),'Performance']+=1
                Factors = Level.loc[Level['Performance']>=2,'Model'].values
            print(Records)
                      
#         kwt = HTML(
#                     value=str(0),
#                     placeholder='Quit Score: ',
#                     description='Quit Score: ',
#                 )
        tar.value=target
        prog1 = FloatProgress(min=start, max=stop,description='Running:')
        prog2 = FloatProgress(min=0, max=100,description='Bootstrapping:')
        MdLs = HTML(
            value=" ",
            placeholder='Models: ',
            description='Models: ',
        )
        MdL = HTML(
            value=" ",
            placeholder='Testing: ',
            description='Testing: ',
        )
        Display (tar,prog1,prog2,MdLs,MdL)
        for j in range(start,stop):
            print(j)
            if j == 1: Inputs = (Combos(XVarriables,j))
            else: Inputs = (Combos(XVarriables,j,Factors))
            MdLs.value=str(len(Inputs))
            i = 0
            procede = True
            for Input in Inputs:
                if FirstRun == False:
#                     print(Records.isin({'Size':[j]}).any().any())
                    if Records.loc[Records['Size']==j].isin({'Number':[i]}).any().any()==True:
                        procede = False
                    else:
                        procede = True
                if procede == True:
                    params = Dense.Params(Scope,target,MP)
                    params['Dpath'] = cwd+'/'+Site+'/'
                    params['Spath'] = params['Dpath']+'/'+target+'/'+str(j)+'_'+str(i)+'/'
                    try:os.mkdir(params['Spath'])
                    except:pass
                    params['Sname'] = 'Y_'
                    params['Inputs'] = Input
                    MdL.value='#'+str(i)+' '+str(params['Inputs'])
                    Display (tar,prog1,prog2,MdLs,MdL)
                    try:
                        print(Level.sort_values('MSE'))
                    except:
                        pass
                    print('Runtime: ',time.time()-Time)
                    params['Model'] = '+'.join(params['Inputs'])
                    RST = RSTF.ReadStandardTimeFill(params,'ECData.csv')#,resample='2H')
                    if target == 'ER':
                        RST.Master = RST.Master.loc[RST.Master['fco2']>0]
                    RST.Scale(params['target'],params['Inputs']) 
                    y = RST.y*1.0
                    X = RST.X*1.0
                    params['N']=int(y.shape[0]/30)
                    params['Memory'] = (math.floor(100/params['proc'])- 5/params['proc']) * .01
                    Y_hat=[]
                    y_true=[]
                    X_true=[]
                    index=[]
                    ones=[]
                    prog2.value=0
#                     if MP == False:
#                         for k in range(params['K']):
#                             Time2 = time.time()
#                             results = Dense.Bootstrap(k,params,X,y)
#                             Y_hat.append(RST.YScaled.inverse_transform(results[0]))
#                             y_true.append(RST.YScaled.inverse_transform(results[1]))
#                             X_true.append(RST.XScaled.inverse_transform(results[2]))
#                             ones.append(results[3])
#                             prog2.value=(k+1)/params['K']*100
#                             print(time.time()-Time2)
#                     else:
                    pool = Pool(processes=processes,maxtasksperchild=75)
                    for k,results in enumerate(pool.imap(partial(Dense.Bootstrap,params=params,X=X,y=y),range(params['K']))):
                        Y_hat.append(RST.YScaled.inverse_transform(results[0]))
                        y_true.append(RST.YScaled.inverse_transform(results[1]))
                        X_true.append(RST.XScaled.inverse_transform(results[2]))
                        ones.append(results[3])
                        prog2.value=(k+1)/params['K']*100
                    pool.close()
                    Y_hat = np.squeeze(np.asanyarray(Y_hat))
                    y_true = np.squeeze(np.asanyarray(y_true))
                    X_true = np.asanyarray(X_true)
                    ones = np.asanyarray(ones)
                    params['Memory'] = .95
#                     if MP == False:
#                         for k in range(1):
#                              mse,se = Dense.Sort_outputs(k,params,Y_hat,y_true,X_true,ones)
#                     else:
                    pool = Pool(processes=1,maxtasksperchild=75)
                    for k,results in enumerate(pool.imap(partial(Dense.Sort_outputs,params=params,
                     Y_hat=Y_hat,y_true=y_true,X_true=X_true,ones=ones),range(1))):
                         mse,se = results
                    pool.close()
                    if i == 0 and FirstRun == True:Level = Stats(mse,se,j,i,params)
                    else:
                        Level = Level.loc[Level['Size']==j]
                        Level = Level.append(Stats(mse,se,j,i,params))
                i += 1
                prog1.value=j+i/len(Inputs)
            Min = Level.loc[Level['MSE']==Level['MSE'].min()]
#             if j ==1:
# #         ch4.Records.loc[fch4.Records['Size']==level-1].sort_values('MSE').reset_index()
#                 Min = Level.loc[Level['MSE']==Level['MSE'].min()]
# #                 Level['T']
# #                 Sv = (((Level['SE']**2+Min['SE'].values[0]**2)/2)**.5)
# #                 Level['T']=(Level['MSE']-Min['MSE'].values[0])/(Sv*(2/params['K'])**.5)
# #                 Level['T2'] = Level['T']
# #                 Level.loc[Level['T']<=stats.t.ppf(1-alpha,params['K']),'Performance']=1
                
#                 Level.loc[Level['MSE']==Min['MSE'].value[0],'Performance']=1
#                 Level.loc[Level['MSE']<=Min['MSE'].value[0]+Min['SE'].value[0],'Performance']+=1
#                 Factors = Level.loc[Level['Performance']==2,'Model'].values
#             else:

#                 Min = Level.loc[Level['MSE']==Level['MSE'].min()]
# #                 Sv = (((Level['SE']**2+Min['SE'].values[0]**2)/2)**.5)
# # #                 Level['T']=(Level['MSE']-Min['MSE'].values[0])/(Sv*(2/params['K'])**.5)
# # #                 Level['T2'] = Level['T']
# # #                 Factors = Level.loc[Level['T']<=1.67,'Model'].values
# # #                 Level.loc[Level['T']<=stats.t.ppf(1-alpha,params['K']),'Performance']=1
                
# #                 Last = Records.loc[Records['Size']==j-1].sort_values('MSE').reset_index()
# #                 Rec = Last.loc[Last['MSE']==Last['MSE'].min()]
                
# #                 Sv = (((Level['SE']**2+Rec['SE'].values[0]**2)/2)**.5)
# #                 Level['T2']=(Level['MSE']-Rec['MSE'].values[0])/(Sv*(2/params['K'])**.5)
                
# #                 Factors2 = Level.loc[Level['T2']<=-1.67,'Model'].values                
            Level.loc[Level['MSE']==Min['MSE'].values[0],'Performance']=1
            Level.loc[Level['MSE']<=Min['MSE'].values[0]+Min['SE'].values[0],'Performance']+=1
            Factors = Level.loc[Level['Performance']>=2,'Model'].values
            if j == 1:
                Records = Level
            else:
                Records = Records.append(Level)
#             kwt.value = str(kwit)
            Records = Records.reset_index(drop=True)
            Records.drop(Records.columns[Records.columns.str.contains('Unnamed',case = False)],axis = 1)
            if Scope == 'Full':
                Records.to_csv(params['Dpath']+'/'+target+'_Runs.csv')
            else:
                Records.to_csv(params['Dpath']+'/'+target+'_Runs2.csv')
            print(Records)
#             if Level['Performance'].max()<2 and j>1:
#                 break

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

                                                                         Model  \
9_0   Wind Spd+Ta+Rn+PPFD+Water Table+Ts 2.5 cm+Ts 15 cm+Active Layer+24H Rain   
9_1  Wind Spd+Ta+PPFD+Water Table+Ts 2.5 cm+Ts 15 cm+VWC+Active Layer+24H Rain   

          MSE  Size  Number        SE  Performance  
9_0  0.597490     9       0  0.047463            2  
9_1  0.602422     9       1  0.062095            1  
Runtime:  10590.468732357025
                                                                           Model  \
0                                                                       Wind Spd   
1                                                                             Ta   
2                                                                             Rn   
3                                                                           PPFD   
4                                                                    Water Table   
5                                                            

In [27]:
       
Level.loc[Level['MSE']==Min['MSE'].values[0],'Performance']=1
Level.loc[Level['MSE']<=Min['MSE'].values[0]+Min['SE'].values[0],'Performance']+=1
print(Min['MSE'].values[0]+Min['SE'].values[0])
print(Min)
print(Level)

0.004155662758921846
                                      Model       MSE  Size  Number        SE  \
7_1  H+Wind Spd+Ta+Rn+PPFD+VWC+Active Layer  0.003723     7       1  0.000433   

     Performance    T        T2  
7_1            1  0.0 -0.684433  
            Model       MSE  Size  Number        SE  Performance
1_0      Wind Spd  0.990742     1       0  0.065715            0
1_1            Ta  1.028427     1       1  0.072491            0
1_2            Rn  0.846922     1       2  0.044759            0
1_3          PPFD  0.783612     1       3  0.037020            0
1_4   Water Table  1.043198     1       4  0.034296            0
1_5     Ts 2.5 cm  1.083033     1       5  0.115067            0
1_6      Ts 15 cm  0.972624     1       6  0.055984            0
1_7           VWC  0.993514     1       7  0.051415            0
1_8  Active Layer  0.943730     1       8  0.078060            0
1_9      24H Rain  1.062332     1       9  0.036843            0


In [12]:
prog1 = FloatProgress(min=1, max=2,description='Running:')
prog2 = FloatProgress(min=0, max=100,description='Bootstrapping:')
MdLs = HTML(
    value=" ",
    placeholder='Models: ',
    description='Models: ',
)
MdL = HTML(
    value=" ",
    placeholder='Testing: ',
    description='Testing: ',
)
Display (tar,prog1,prog2,MdLs,MdL,kwt)

i = 99
j = 99
target = 'fco2'
Name ='ER_Runs'


params = Dense.Params(Scope,target,MP)
params['Dpath'] = cwd+'/'+Site+'/'
params['Spath'] = params['Dpath']+target+'/'+str(j)+'_'+str(i)+'/'
try:
    os.mkdir(params['Spath'])
except:
    pass
params['Sname'] = 'Y_'
params['Inputs'] = ['PPFD_Avg','VWC','VPD','Shrub']#Input
# params['Inputs'] = ['wind_speed', 'PPFD_Avg', 'VPD', 'AirTC_Avg','VWC','Temp',
#               'Sedge','Grass','Shrub','Sparse','Upland']
# params['Inputs'] = ['wind_speed', 'PPFD_Avg', 'AirTC_Avg','VWC','Temp',
#               'Sedge','Grass','Shrub','Sparse','Upland']


MdL.value='#'+str(i)+' '+str(params['Inputs'])
Display (tar,prog1,prog2,MdLs,MdL,kwt)
params['Model'] = '+'.join(params['Inputs'])
RST = RSTF.ReadStandardTimeFill(params,'ECData.csv')#,resample='2H')

if Name =='ER_Runs':
    RST.Master = RST.Master.loc[RST.Master['fco2']>0]
# if Name =='ER_Runs':
#     RST.Master = RST.Master.loc[RST.Master['PPFD_Avg']<10]
RST.Scale(params['target'],params['Inputs']) 
y = RST.y*1.0
X = RST.X*1.0
params['N']=int(y.shape[0]/30)
params['Memory'] = (math.floor(100/params['proc'])- 5/params['proc']) * .01
Y_hat=[]
y_true=[]
X_true=[]
index=[]
ones=[]
prog2.value=0
if MP == False:
    for k in range(params['K']):
        Time2 = time.time()
        results = Dense.Bootstrap(k,params,X,y)
        Y_hat.append(RST.YScaled.inverse_transform(results[0]))
        y_true.append(RST.YScaled.inverse_transform(results[1]))
        X_true.append(RST.XScaled.inverse_transform(results[2]))
        ones.append(results[3])
        prog2.value=(k+1)/params['K']*100
        print(time.time()-Time2)
else:
    pool = Pool(processes=processes,maxtasksperchild=75)
    for k,results in enumerate(pool.imap(partial(Dense.Bootstrap,params=params,X=X,y=y),range(params['K']))):
        Y_hat.append(RST.YScaled.inverse_transform(results[0]))
        y_true.append(RST.YScaled.inverse_transform(results[1]))
        X_true.append(RST.XScaled.inverse_transform(results[2]))
        ones.append(results[3])
        prog2.value=(k+1)/params['K']*100
    pool.close()
Y_hat = np.squeeze(np.asanyarray(Y_hat))
y_true = np.squeeze(np.asanyarray(y_true))
X_true = np.asanyarray(X_true)
ones = np.asanyarray(ones)
params['Memory'] = .95
if MP == False:
    for k in range(1):
         mse,se = Dense.Sort_outputs(k,params,Y_hat,y_true,X_true,ones)
else:
    pool = Pool(processes=1,maxtasksperchild=75)
    for k,results in enumerate(pool.imap(partial(Dense.Sort_outputs,params=params,
     Y_hat=Y_hat,y_true=y_true,X_true=X_true,ones=ones),range(1))):
         mse,se = results
    pool.close()
    
    
Level = Stats(mse,se,j,i,params)
Level.to_csv(params['Dpath']+Name+'.csv')

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

KeyError: "['PPFD_Avg' 'VPD' 'Shrub'] not in index"

In [17]:
# print(params)

Level.to_csv(params['Dpath']+Name+'.csv')
# Level = Stats(mse,se,j,i,params)
# Level.to_csv(params['Dpath']+target+'_Final_Runs.csv')

# plt.figure()
# print(RST.Master
#      )

# pool = Pool(processes=1,maxtasksperchild=75)
# for k,results in enumerate(pool.imap(partial(Dense.Sort_outputs,params=params,
#  Y_hat=Y_hat,y_true=y_true,X_true=X_true,ones=ones),range(1))):
#      mse,se = results
# pool.close()

# plt.scatter(Y_hat,y_true)
# print(metrics.mean_squared_error(y_true,Y_hat))
# print(metrics.r2_score(y_true,Y_hat))

# plt.figure()

# plt.plot(Y_hat)
# plt.plot(y_true)

# print(Y_hat.shape,y_true.shape)
print(Level)

                        Model       MSE  Size  Number        SE  Performance
99_99  PPFD_Avg+VWC+VPD+Shrub  0.011892    99      99  0.002806            0


# Predict

In [ ]:

plt.figure(figsize=(8,7))
Data = Data.sort_values(by=Key)
# Data.index = Data[Key]

Data = Data[np.isfinite(Data['True'])]

plt.scatter(RST.Master[Key],RST.Master[target],edgecolor='black',facecolor='white')
plt.plot(Data[Key],Data[target],
         label= params['target']+' Model\nr^2: '+str(np.round(metrics.r2_score(Data['True'],
                                                                   Data[params['target']])**2,3)))
# plt.plot(Data.index,Data['Var'],label= params['target']+
# ' Model\nRMSE: '+str(np.round(metrics.mean_squared_error(Data['True'],
#                                                                    Data[params['target']])**2,3)))


plt.fill_between(Data[Key], Data[target]-Data['PI'], 
                 Data[target]+Data['PI'],  color = 'green', alpha = 0.4, 
                 label = '95% PI')
plt.fill_between(Data[Key], Data[target]-Data['CI'], 
                 Data[target]+Data['CI'],  color = 'red', alpha = 0.4, 
                 label = '95% CI')
plt.legend()
# print(RST.Master)

plt.figure()

# plt.plot(Data[Key], Data['Var'])

plt.scatter(Data[target],Data['True'])

# print(Data['SE'])